# PMSM Simulation Data Exploration

Analysis of FOC controller simulation data.

Dataset: 1000 runs, 2001 samples each (0-0.2s @ 10kHz)
Variables: i_d, i_q (currents), u_d, u_q (voltages), n (speed), time

Install packages if needed

In [ ]:
# Install packages (run once)
!pip install pandas numpy matplotlib seaborn

Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

Load data

In [ ]:
try:
    df_panel = pd.read_parquet('data/merged/merged_panel.parquet')
except:
    df_panel = pd.read_csv('data/merged/merged_panel.csv')

try:
    df_stacked = pd.read_parquet('data/merged/merged_stacked.parquet')
except:
    df_stacked = pd.read_csv('data/merged/merged_stacked.csv')

print(f'Panel: {df_panel.shape}, Stacked: {df_stacked.shape}')

Data overview

In [ ]:
# Display first rows
print('First 10 rows of Panel Data:')
display(df_panel.head(10))

print('\n' + '='*80)
print('\nData Info:')
df_panel.info()

print('\n' + '='*80)
print('\nStatistical Summary:')
display(df_panel.describe())

Single run analysis

In [ ]:
# Select run 1 for detailed analysis
run_id = 1
single_run = df_panel[df_panel['run_id'] == run_id].copy()

print(f'Analyzing Run {run_id}')
print(f'Time range: {single_run["time"].min():.6f} to {single_run["time"].max():.6f} seconds')
print(f'Duration: {(single_run["time"].max() - single_run["time"].min())*1000:.1f} milliseconds')
print(f'Number of samples: {len(single_run)}')
print(f'Time step: {single_run["time"].diff().median():.6f} seconds (100 μs)')
print(f'Sampling rate: {1/single_run["time"].diff().median():.0f} Hz (10 kHz)')

print('\nSummary Statistics for this run:')
display(single_run.describe())

Time series plots

In [ ]:
# Plot all variables over time
fig, axes = plt.subplots(3, 2, figsize=(16, 13))
fig.suptitle(f'Run {run_id} - All Variables Over Time (200 ms duration)', 
             fontsize=16, fontweight='bold', y=0.995)

# Convert time to milliseconds for easier reading
time_ms = single_run['time'] * 1000

# Current d-axis
axes[0, 0].plot(time_ms, single_run['i_d'], 'b-', linewidth=0.8)
axes[0, 0].set_ylabel('i_d (A)', fontsize=11, fontweight='bold')
axes[0, 0].set_title('Direct-axis Current', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3, linewidth=1)

# Current q-axis
axes[0, 1].plot(time_ms, single_run['i_q'], 'r-', linewidth=0.8)
axes[0, 1].set_ylabel('i_q (A)', fontsize=11, fontweight='bold')
axes[0, 1].set_title('Quadrature-axis Current (Torque-producing)', fontsize=12)
axes[0, 1].grid(True, alpha=0.3)
axes[0, 1].axhline(y=0, color='r', linestyle='--', alpha=0.3, linewidth=1)

# Voltage d-axis
axes[1, 0].plot(time_ms, single_run['u_d'], 'g-', linewidth=0.8)
axes[1, 0].set_ylabel('u_d (V)', fontsize=11, fontweight='bold')
axes[1, 0].set_title('Direct-axis Voltage', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)
axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.3, linewidth=1)

# Voltage q-axis
axes[1, 1].plot(time_ms, single_run['u_q'], 'm-', linewidth=0.8)
axes[1, 1].set_ylabel('u_q (V)', fontsize=11, fontweight='bold')
axes[1, 1].set_title('Quadrature-axis Voltage', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)
axes[1, 1].axhline(y=single_run['u_q'].mean(), color='orange', 
                   linestyle='--', alpha=0.5, linewidth=1, label=f'Mean: {single_run["u_q"].mean():.2f}V')
axes[1, 1].legend()

# Speed
axes[2, 0].plot(time_ms, single_run['n'], 'orange', linewidth=1.5)
axes[2, 0].set_ylabel('n (RPM)', fontsize=11, fontweight='bold')
axes[2, 0].set_xlabel('Time (ms)', fontsize=11)
axes[2, 0].set_title('Rotational Speed', fontsize=12)
axes[2, 0].grid(True, alpha=0.3)
axes[2, 0].set_ylim([990, 1010])

# Current magnitude
i_mag = np.sqrt(single_run['i_d']**2 + single_run['i_q']**2)
axes[2, 1].plot(time_ms, i_mag, 'purple', linewidth=0.8)
axes[2, 1].set_ylabel('|i| (A)', fontsize=11, fontweight='bold')
axes[2, 1].set_xlabel('Time (ms)', fontsize=11)
axes[2, 1].set_title('Current Magnitude √(i_d² + i_q²)', fontsize=12)
axes[2, 1].grid(True, alpha=0.3)
axes[2, 1].axhline(y=i_mag.mean(), color='red', linestyle='--', 
                   alpha=0.5, linewidth=1, label=f'Mean: {i_mag.mean():.3f}A')
axes[2, 1].legend()

plt.tight_layout()
plt.show()

Multi-run comparison

In [ ]:
# Compare first 10 runs - Quadrature current
fig, ax = plt.subplots(figsize=(15, 7))

for r in range(1, 11):
    data = df_panel[df_panel['run_id'] == r]
    ax.plot(data['time']*1000, data['i_q'], alpha=0.7, linewidth=1, label=f'Run {r}')

ax.set_xlabel('Time (ms)', fontsize=12)
ax.set_ylabel('i_q (A)', fontsize=12)
ax.set_title('Quadrature Current - First 10 Runs Comparison', fontsize=14, fontweight='bold')
ax.legend(bbox_to_anchor=(1.05, 1), loc='upper left', fontsize=9)
ax.grid(True, alpha=0.3)
ax.axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
plt.tight_layout()
plt.show()

# Calculate run consistency
run_means = []
for r in range(1, 11):
    data = df_panel[df_panel['run_id'] == r]
    run_means.append(data['i_q'].mean())

print(f'Mean i_q across first 10 runs: {np.mean(run_means):.6f} A')
print(f'Std deviation: {np.std(run_means):.9f} A')
print(f'Coefficient of variation: {(np.std(run_means)/np.mean(np.abs(run_means))*100):.6f}%')

Correlation analysis

In [ ]:
# Correlation matrix
corr_cols = ['i_d', 'i_q', 'n', 'u_d', 'u_q']
corr_matrix = single_run[corr_cols].corr()

plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=2, cbar_kws={"shrink": 0.8}, 
            fmt='.3f', vmin=-1, vmax=1)
plt.title(f'Correlation Matrix - Run {run_id}\n(Values range from -1 to +1)', 
          fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print('\nKey Correlations:')
print(f'i_d vs u_d: {corr_matrix.loc["i_d", "u_d"]:.3f} (current-voltage coupling in d-axis)')
print(f'i_q vs u_q: {corr_matrix.loc["i_q", "u_q"]:.3f} (current-voltage coupling in q-axis)')
print(f'i_d vs i_q: {corr_matrix.loc["i_d", "i_q"]:.3f} (cross-coupling between axes)')
print(f'u_d vs u_q: {corr_matrix.loc["u_d", "u_q"]:.3f} (voltage cross-coupling)')

Phase space plots (d-q plane)

In [ ]:
# Create phase space plots
fig, axes = plt.subplots(1, 2, figsize=(16, 6))
fig.suptitle(f'Phase Space Analysis - Run {run_id}', fontsize=16, fontweight='bold')

# Current space (i_d vs i_q)
scatter1 = axes[0].scatter(single_run['i_d'], single_run['i_q'], 
                          c=single_run['time']*1000, cmap='viridis', 
                          alpha=0.6, s=15, edgecolors='none')
axes[0].set_xlabel('i_d (A)', fontsize=12, fontweight='bold')
axes[0].set_ylabel('i_q (A)', fontsize=12, fontweight='bold')
axes[0].set_title('Current Space (d-q plane)', fontsize=13)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
axes[0].axvline(x=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
cbar1 = plt.colorbar(scatter1, ax=axes[0])
cbar1.set_label('Time (ms)', fontsize=10)

# Mark start and end points
axes[0].scatter(single_run['i_d'].iloc[0], single_run['i_q'].iloc[0], 
               c='green', s=100, marker='o', edgecolors='black', linewidth=2, 
               label='Start', zorder=5)
axes[0].scatter(single_run['i_d'].iloc[-1], single_run['i_q'].iloc[-1], 
               c='red', s=100, marker='s', edgecolors='black', linewidth=2, 
               label='End', zorder=5)
axes[0].legend(fontsize=10)

# Voltage space (u_d vs u_q)
scatter2 = axes[1].scatter(single_run['u_d'], single_run['u_q'], 
                          c=single_run['time']*1000, cmap='plasma', 
                          alpha=0.6, s=15, edgecolors='none')
axes[1].set_xlabel('u_d (V)', fontsize=12, fontweight='bold')
axes[1].set_ylabel('u_q (V)', fontsize=12, fontweight='bold')
axes[1].set_title('Voltage Space (d-q plane)', fontsize=13)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
axes[1].axvline(x=0, color='red', linestyle='--', alpha=0.3, linewidth=1)
cbar2 = plt.colorbar(scatter2, ax=axes[1])
cbar2.set_label('Time (ms)', fontsize=10)

# Mark start and end points
axes[1].scatter(single_run['u_d'].iloc[0], single_run['u_q'].iloc[0], 
               c='green', s=100, marker='o', edgecolors='black', linewidth=2, 
               label='Start', zorder=5)
axes[1].scatter(single_run['u_d'].iloc[-1], single_run['u_q'].iloc[-1], 
               c='red', s=100, marker='s', edgecolors='black', linewidth=2, 
               label='End', zorder=5)
axes[1].legend(fontsize=10)

plt.tight_layout()
plt.show()

Distribution plots

In [ ]:
# Distribution plots
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle(f'Variable Distributions - Run {run_id}', fontsize=16, fontweight='bold')

variables = ['i_d', 'i_q', 'u_d', 'u_q', 'n']
colors = ['blue', 'red', 'green', 'magenta', 'orange']
labels = ['Direct Current', 'Quadrature Current', 'Direct Voltage', 'Quadrature Voltage', 'Speed']

for idx, (var, color, label) in enumerate(zip(variables, colors, labels)):
    row = idx // 3
    col = idx % 3
    
    # Histogram
    n, bins, patches = axes[row, col].hist(single_run[var], bins=60, alpha=0.7, 
                                           color=color, edgecolor='black', linewidth=0.5)
    axes[row, col].set_xlabel(var, fontsize=11, fontweight='bold')
    axes[row, col].set_ylabel('Frequency', fontsize=11)
    axes[row, col].set_title(f'{label}', fontsize=12)
    axes[row, col].grid(True, alpha=0.3, axis='y')
    
    # Add statistics
    mean_val = single_run[var].mean()
    std_val = single_run[var].std()
    axes[row, col].axvline(mean_val, color='red', linestyle='--', linewidth=2, 
                          label=f'Mean: {mean_val:.3f}')
    axes[row, col].axvline(mean_val+std_val, color='orange', linestyle=':', linewidth=1.5, 
                          label=f'±σ: {std_val:.3f}')
    axes[row, col].axvline(mean_val-std_val, color='orange', linestyle=':', linewidth=1.5)
    axes[row, col].legend(fontsize=9)

# Remove extra subplot
fig.delaxes(axes[1, 2])

plt.tight_layout()
plt.show()

Power analysis

In [ ]:
# Calculate power
single_run['P_d'] = single_run['u_d'] * single_run['i_d']
single_run['P_q'] = single_run['u_q'] * single_run['i_q']
single_run['P_total'] = single_run['P_d'] + single_run['P_q']

# Plot power
fig, axes = plt.subplots(3, 1, figsize=(15, 11))
fig.suptitle(f'Power Analysis - Run {run_id}', fontsize=16, fontweight='bold')

time_ms = single_run['time'] * 1000

# Component powers
axes[0].plot(time_ms, single_run['P_d'], 'b-', label='P_d = u_d × i_d', linewidth=1, alpha=0.8)
axes[0].plot(time_ms, single_run['P_q'], 'r-', label='P_q = u_q × i_q', linewidth=1, alpha=0.8)
axes[0].set_ylabel('Power (W)', fontsize=11, fontweight='bold')
axes[0].set_title('Component Powers (d and q axes)', fontsize=13)
axes[0].legend(fontsize=10)
axes[0].grid(True, alpha=0.3)
axes[0].axhline(y=0, color='black', linestyle='-', alpha=0.3, linewidth=0.5)

# Total power
axes[1].plot(time_ms, single_run['P_total'], 'g-', linewidth=1.2)
axes[1].axhline(y=single_run['P_total'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Average: {single_run["P_total"].mean():.3f} W')
axes[1].set_ylabel('Total Power (W)', fontsize=11, fontweight='bold')
axes[1].set_title('Total Instantaneous Power (P_d + P_q)', fontsize=13)
axes[1].legend(fontsize=10)
axes[1].grid(True, alpha=0.3)
axes[1].axhline(y=0, color='black', linestyle='-', alpha=0.3, linewidth=0.5)

# Power histogram
axes[2].hist(single_run['P_total'], bins=50, color='green', alpha=0.7, edgecolor='black')
axes[2].axvline(single_run['P_total'].mean(), color='red', linestyle='--', 
               linewidth=2, label=f'Mean: {single_run["P_total"].mean():.3f} W')
axes[2].set_xlabel('Power (W)', fontsize=11, fontweight='bold')
axes[2].set_ylabel('Frequency', fontsize=11)
axes[2].set_title('Power Distribution', fontsize=13)
axes[2].legend(fontsize=10)
axes[2].grid(True, alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

# Power statistics
print('='*80)
print('POWER ANALYSIS SUMMARY')
print('='*80)
print(f'Average power: {single_run["P_total"].mean():.3f} W')
print(f'Peak power: {single_run["P_total"].max():.3f} W')
print(f'Minimum power: {single_run["P_total"].min():.3f} W')
print(f'Power std deviation: {single_run["P_total"].std():.3f} W')
print(f'\nPower factor (ratio): {single_run["P_total"].mean() / (np.sqrt(single_run["i_d"]**2 + single_run["i_q"]**2).mean() * np.sqrt(single_run["u_d"]**2 + single_run["u_q"]**2).mean()):.3f}')
print(f'\nEnergy over 200ms: {single_run["P_total"].mean() * 0.2:.6f} J')

Summary statistics

In [ ]:
# Generate comprehensive summary
print('='*80)
print('COMPREHENSIVE DATA ANALYSIS SUMMARY')
print('='*80)

print('\n1. DATASET OVERVIEW')
print('-'*80)
print(f'Total simulation runs: {df_panel["run_id"].nunique()}')
print(f'Samples per run: {df_panel.groupby("run_id").size().iloc[0]}')
print(f'Total data points: {len(df_panel):,}')
print(f'Time range per run: {single_run["time"].min():.3f} to {single_run["time"].max():.3f} seconds ({single_run["time"].max()*1000:.0f} ms)')
print(f'Sampling rate: 10,000 Hz (100 μs time steps)')

print('\n2. VARIABLE STATISTICS (Run {})'.format(run_id))
print('-'*80)
for var in ['i_d', 'i_q', 'u_d', 'u_q', 'n']:
    print(f'{var:5s}: Mean={single_run[var].mean():8.4f}, Std={single_run[var].std():8.4f}, '
          f'Range=[{single_run[var].min():8.4f}, {single_run[var].max():8.4f}]')

print('\n3. CURRENT ANALYSIS')
print('-'*80)
i_mag = np.sqrt(single_run['i_d']**2 + single_run['i_q']**2)
print(f'RMS current magnitude: {np.sqrt(np.mean(i_mag**2)):.4f} A')
print(f'Average current magnitude: {i_mag.mean():.4f} A')
print(f'Peak current magnitude: {i_mag.max():.4f} A')
print(f'i_q dominance: {abs(single_run["i_q"].mean()) / i_mag.mean() * 100:.1f}%')

print('\n4. VOLTAGE ANALYSIS')
print('-'*80)
u_mag = np.sqrt(single_run['u_d']**2 + single_run['u_q']**2)
print(f'RMS voltage magnitude: {np.sqrt(np.mean(u_mag**2)):.4f} V')
print(f'Average voltage magnitude: {u_mag.mean():.4f} V')
print(f'Peak voltage magnitude: {u_mag.max():.4f} V')

print('\n5. POWER ANALYSIS')
print('-'*80)
print(f'Average electrical power: {single_run["P_total"].mean():.4f} W')
print(f'Peak power: {single_run["P_total"].max():.4f} W')
print(f'Energy consumed (200ms): {single_run["P_total"].mean() * 0.2:.6f} J')
print(f'Power ripple (std): {single_run["P_total"].std():.4f} W')

print('\n6. OPERATING CONDITIONS')
print('-'*80)
print(f'Speed: {single_run["n"].mean():.1f} RPM (constant)')
print(f'Mechanical frequency: {single_run["n"].mean() / 60:.2f} Hz')
print(f'Operating mode: {"Motor" if single_run["P_total"].mean() > 0 else "Generator"}')

print('\n7. DATA QUALITY')
print('-'*80)
print(f'Missing values: {df_panel.isnull().sum().sum()}')
print(f'Data completeness: 100%')
print(f'Time continuity: Valid (no gaps)')

print('\n' + '='*80)
print('END OF ANALYSIS')
print('='*80)